# Compare posteriors.

This notebook loads up posterior distributions for different shoreline options and visualizes them with corner plots. 

In [ ]:
from shoreline import * 

In [ ]:
import arviz as az 
import corner 

print('Here are the available posteriors:')
posteriors = {}
for f in glob.glob('posteriors/*-numpyro.nc'):
    k = f.split('/')[1].split('-numpyro.nc')[0]
    print(f' {k}')
    posteriors[k] = az.from_netcdf(f)
    

$$ \log_{10} f_{\sf shoreline}  =  \log_{10} f_{\sf 0} + p \cdot  \log_{10} \left(\frac{v_{\sf esc}}{v_{\sf esc, \oplus}}\right) + q \cdot  \log_{10} \left(\frac{L_\star}{L_\odot}\right)$$

The intersection with the habitable zone is roughly $f_{\sf shoreline} = 1$, $v_{\sf esc}/v_{\sf esc, \oplus}=1$:

$$ - \frac{\log_{10} f_{\sf 0}}{q} =    \log_{10} \left(\frac{L_\star}{L_\odot}\right)$$


Print out parameter values + confidence intervals.

In [ ]:
var_names = ["log_f_0", "p", "q", "ln_w"] 
to_latex = {"log_f_0":'\log_{10} f_0', 
            "p":"p", 
            "q":"q", 
            "ln_w":"\ln w"} 

func_dict = {
    "median": np.median,
    "lower": lambda x: np.median(x) - np.percentile(x, 50-68.3/2), 
    "upper": lambda x: np.percentile(x, 50+68.3/2) - np.median(x) }

def create_latex_name(s):
    return clean(s).replace('0', 'o')

for kind, inference in posteriors.items():
    print(kind)

    po = inference.posterior
    po['logL_nohz'] = -po['log_f_0']/po['q']
    po['L_nohz'] = 10**po['logL_nohz']

    summary = az.summary(inference,  stat_funcs=func_dict) #var_names=var_names + ['w'],
    for k in summary['median'].keys():
        lower, upper = summary["lower"][k], summary["upper"][k] 
        symmetric = np.abs(lower - upper)/(lower + upper) < 0.1
        l = to_latex.get(k, k)
        if symmetric:
            sigma = (lower + upper)/2
            s = f'{summary["median"][k]:.3g} \pm {sigma:.2g}'
        else:
            s = f'{summary["median"][k]:.3g}_{{-{lower:.2g}}}^{{+{upper:.2g}}}'
        
        print(rf'\newcommand{{\{create_latex_name(k)}}}{{{s}}}')


    print()

In [ ]:
 1.0 - np.exp(-0.5 * np.arange(0.5, 2.1, 0.5) ** 2)

In [ ]:
from scipy.special import erf
levels = erf(np.arange(1,3)/np.sqrt(2))
levels

In [ ]:
%matplotlib widget

In [ ]:
posteriors.keys()

In [ ]:
plt.close("all")
figsize=(6,6)
question = {"any": "Any atmosphere or surface volatiles?", "CO2": "Warm CO$_2$ atmosphere?"}
better_titles = ["$\sf \log_{10}f_0$", "$\sf p$", "$\sf q$", "$\sf \ln w$"]

kw = dict(
    var_names=var_names,
    labels=better_titles,
    bins=50,
    levels=levels,
    plot_density=False,
    plot_datapoints=False,
    show_titles=True,
    title_kwargs=dict(fontsize=9),
    range=[[-1, 6], [0, 10], [0, 3], [-6, 2]],
)
for kind in ["any", "CO2"]:
    labels = {
        "all": f"Exoplanets + Solar System",
        "exo": "Exoplanets Only",
        "solar": "Solar System Only",
    }
    fig = plt.figure(figsize=figsize)
    colors = {
        f"solar-{kind}": "mediumslateblue",
        f"exo-{kind}": "tomato",
        f"all-{kind}": "black",
    }
    for k in colors:
        if "all" in k:
            alpha = 1
        else:
            alpha = 0.5
        corner.corner(
            posteriors[f'{k}-uncertainties=True'],
            color=colors[k],
            hist_kwargs=dict(density=True, label=labels[k.split("-")[0]]),
            fig=fig,
            contour_kwargs=dict(alpha=alpha, color=colors[k]),
            **kw,
        )
    plt.sca(fig.get_axes()[0])
    plt.legend(bbox_to_anchor=(1, 1), frameon=False)
    plt.sca(fig.get_axes()[2])
    plt.title(question[kind])
    #for ax in fig.get_axes():
    #    ax.title.set_size(10)
        #title = ax.get_title()
        #if title != '':
        #    ax.set_title(title.replace('=', '=\n'))
    plt.savefig(f"figures/posteriors-{kind}.pdf")

In [ ]:
fig = plt.figure(figsize=figsize)
subset = 'all'
for kind in ["any", "CO2"]:
    kind_label = {"any": 'any atmosphere', 
                  "CO2":'CO$_2$ atmosphere'}
    uncertainty_labels = {True:f'with uncertainties', 
                          False:f'without uncertainties'}
    
    colors = {f"any": "green", "CO2": "orchid"}
    # corner.corner(posteriors[f'all-{kind}'], color=colors[kind], hist_kwargs=dict(density=True, label=kind), fig=fig, **kw)
    for u in [True, False]:
        alpha = {True: 1.0, False: 0.3}[u]
        color_kwargs = dict(alpha=alpha, color=colors[kind])
        corner.corner(
            posteriors[f"{subset}-{kind}-uncertainties={u}"],
            color=colors[kind],
            contour_kwargs=color_kwargs,
            hist_kwargs=color_kwargs | dict(density=True, label=f'{kind_label[kind]}, {uncertainty_labels[u]}'),
            fig=fig,
            **(kw | dict(show_titles = False)),
        )
    plt.sca(fig.get_axes()[0])
    plt.legend(bbox_to_anchor=(1, 1), frameon=False)
    plt.sca(fig.get_axes()[0])
    plt.legend(bbox_to_anchor=(1, 1), frameon=False)
    plt.savefig('figures/posteriors-with-and-without-uncertainties.pdf')

In [ ]:
levels

Look at exoplanets-only, solar-system-only, everything together.
I think I have to keep in the H-atmopsheres to reign in the 